# HMM with TensorFlow

In [1]:
import tensorflow as tf
import numpy as np
import sys

## Weather HMM example
Das Wetter bei deinem Übersee-Chatfreund lässt sich durch eine Markowkette $X_1,X_2,\ldots$ mit
Zustandsraum $Q=\{\texttt{sun},\texttt{rain},\texttt{storm}\}$ und Übergangsmatrix
$$A=(A[r,s])_{\scriptsize r,s \in Q} = \begin{pmatrix}
0.7 & 0.2 & 0.1\\
0.3 & 0.5 & 0.2\\
0.2 & 0.6 & 0.2\\
\end{pmatrix}
$$
beschreiben (Reihen und Spalten in der Reihenfolge `sun`, `rain`, `storm`).

Dabei sei $X_i$ das Wetter am $i$-ten Tag und $X_1=\texttt{sun}$. Dein Freund verfolgt vom Wetter abhängig Aktivitäten entweder drinnen 
(`in`) oder draußen (`out`). Sei $\Sigma:=\{\texttt{in},\texttt{out}\}$. Folgende Matrix
beschreibe die vom Wetter abhängenden Wahrscheinlichkeiten (Wkeiten) der Aktivitäten
$$B=\big(B[q,s]\big)_{\scriptsize\begin{array}{l}q\in Q\\s\in \Sigma\end{array}} = \begin{pmatrix}
0.4 & 0.6 \\
0.8 & 0.2 \\
0.9 & 0.1 \\
\end{pmatrix}
.$$ 
Ablesebeispiel: Dein Freund bleibt mit Wkeit 0.9 drinnen, wenn es an dem Tag stürmt (Spalten in der Reihenfolge `in`, `out`).
Beantworte folgende Fragen für das durch $Q,\Sigma,A,B$ und $X_1$ gegebene Hidden-Markow-Modell.
Was ist die Wkeit 
$$P(Y_1=Y_2=Y_3=\texttt{in}),$$

dass dein Freund am allen drei Tagen drinnen bleibt?

![forward DP table](forwardManually.png)

**Solution: P(Y=y) = 0.1308**

## Specify the Model

### Example Model Parameters

In [2]:
n = 3 # number of states
s = 2 # emission alphabet size

In [3]:
A_init = np.array([[7, 2, 1], [3, 5, 2], [2, 6, 2]]) / 10.0
B_init = np.array([[4, 6], [8, 2], [9, 1]]) / 10.0
X1_dist = np.array([1., 0., 0.]) # starts with sun
n, s = B_init.shape # number of states, emission alphabet size
y = np.array([0, 0, 0]) # in, in, in

In [4]:
print("transitions:\n", A_init, "\nemissions:\n", B_init)

transitions:
 [[0.7 0.2 0.1]
 [0.3 0.5 0.2]
 [0.2 0.6 0.2]] 
emissions:
 [[0.4 0.6]
 [0.8 0.2]
 [0.9 0.1]]


#### Forward recursion
$$\alpha[i,q] = B[q, y[i]] \sum_{q'} \alpha[i-1, q'] \cdot A[q',q] $$

In [5]:
# tf variants of the transition and emission matrix
A = tf.Variable(A_init, trainable = True)
B = tf.Variable(B_init, trainable = True)

### Forward Variables and Algorithm
$$ \alpha(q,i) = \sum_{x_1,\ldots, x_{i-1}\in Q} P(x_1,\ldots, x_{i-1}, X_i=q, y_1,\ldots, y_i)$$
Initialization: 
$$ \alpha(q, 1) = \sum_{q\in Q} P(X_1 = q)\cdot B[q,y[0]]$$

In [6]:
def forward(y # observation sequence
           ):
    """ Forward Algorithm for Computing Sequence Likelihood """
    ell = y.shape[0]
    α = tf.Variable(np.zeros([ell, n]), trainable = False)
    
    # initialization
    α[0].assign(tf.multiply(B[:, y[0]], X1_dist))
    
    # forward algorithm
    for i in range(1, ell):
        # compute i-th row of DP table
        R = tf.linalg.matvec(A, α[i-1], transpose_a = True)
        α[i].assign(tf.multiply(B[:, y[i]], R))
    return α

def emiProb(α):
    return np.sum(α[-1,:])

In [7]:
α = forward(y)
α.numpy()

array([[0.4    , 0.     , 0.     ],
       [0.112  , 0.064  , 0.036  ],
       [0.04192, 0.0608 , 0.02808]])

In [8]:
Py = emiProb(α)
Py

0.1308

## A HMM as a Special Case of a Recurrent Neural Network
We use the notation of RNNs similar to that in [Dive into Deep Learning](https://d2l.ai/chapter_recurrent-neural-networks/bptt.html). $h_t$ is a size $n$ vector of RNN-"hidden states" (these are real numbers, not to be confused with the hidden states of HMMs, which are from $Q$).  
$$ h_t = f(x_t, h_{t-1}; A, B)$$
We chose the outputs
$$ o_t = \text{sum}(h_t) = h_t[0] + \cdots + h_t[n-1] \in [0,1]$$
so that the final output $o_T$ is just the likelihood of the sequence $P(Y)$.
This RNN does not need to produce intermediate outputs $o_t$ for $t<T$ as they are not used yet. However, they could be used in conjunction with a backwards pass.

### HMMCell
As a template we use the code for [tf.keras.layers.SimpleRNNCell](https://github.com/tensorflow/tensorflow/blob/v2.4.1/tensorflow/python/keras/layers/recurrent.py#L1222-L1420)

In [9]:
from HMMCell import HMMCell

## Test the HMM cell

In [10]:
print (f"n={n}, s={s}")

A_init = np.array([[7, 2, 1], [3, 5, 2], [2, 6, 2]]) / 10.0
B_init = np.array([[4, 6], [8, 2], [9, 1]]) / 10.0
I_init = np.array([1, 1e-10, 1e-10]) # start with X1=sun (very likely)

# take ln to cancel out the softmax that is applied to obtain a stochastic matrix 
A_init = np.log(A_init)
B_init = np.log(B_init)
I_init = np.log(I_init)

A_initializer = tf.keras.initializers.Constant(A_init)
B_initializer = tf.keras.initializers.Constant(B_init)
I_initializer = tf.keras.initializers.Constant(I_init)

yi = np.array([[1., 0]]).astype(np.float32) # np.random.random([batch_size, s]).astype(np.float32)
states = np.array([[.4, 0, 0]]).astype(np.float32) # np.random.random([batch_size, n]).astype(np.float32)
hmmC = HMMCell(n,
               transition_initializer=A_initializer,
               emission_initializer=B_initializer,
               init_initializer=I_initializer
              )

output = hmmC(yi, [0, states, 0.])
print("output:\n", output[0], "\n")

A = hmmC.A
B = hmmC.B
I = hmmC.I

with np.printoptions(precision=5, suppress=True):
    print("transition matrix A:\n", A.numpy())
    print("emission matrix B:\n", B.numpy())
    print("initial distribution I:\n", I.numpy())


n=3, s=2
output:
 [[<tf.Tensor: shape=(1,), dtype=int8, numpy=array([0], dtype=int8)>, <tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.112, 0.064, 0.036]], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.212], dtype=float32)>]] 

transition matrix A:
 [[0.7 0.2 0.1]
 [0.3 0.5 0.2]
 [0.2 0.6 0.2]]
emission matrix B:
 [[0.4 0.6]
 [0.8 0.2]
 [0.9 0.1]]
initial distribution I:
 [1. 0. 0.]


In [11]:
inputs = np.array([[[1, 0],[1, 0],[1, 0]]]).astype(np.float32)
hmm = tf.keras.layers.RNN(hmmC, return_sequences = True, return_state = True)
  
alpha, _, lastcol, loglik = hmm(inputs)
alpha = alpha[1]

print()
with np.printoptions(precision=5, suppress=True):
    print ("α=\n", alpha.numpy(),
       "\nlast column of forward table:", lastcol.numpy(),
      "\nlikelihood=", loglik.numpy())


α=
 [[[0.4     0.      0.     ]
  [0.112   0.064   0.036  ]
  [0.04192 0.0608  0.02808]]] 
last column of forward table: [[0.04192 0.0608  0.02808]] 
likelihood= [0.1308]


### Training

In [12]:
from dishonest_casino import get_casino_dataset
batch_size = 128
ds = get_casino_dataset().repeat().batch(batch_size)

for inputs in ds.take(1):
    pass
#inputs

In [13]:
n=2
dcc = HMMCell(n)
Q = dcc(inputs[:,0,:], [tf.zeros(batch_size, dtype=tf.int8),
                    tf.zeros([batch_size, s], dtype=tf.float32), 
                    tf.zeros(n, dtype=tf.float32)])

In [14]:
# the model
F = tf.keras.layers.RNN(dcc, return_state = True)

In [15]:
def print_pars(cell):
    with np.printoptions(precision=5, suppress=True):
        print("transition matrix A:\n", cell.A.numpy())
        print("emission matrix B:\n", cell.B.numpy())
        print("initial distribution I:\n", cell.I.numpy())

In [16]:
print_pars(dcc)

transition matrix A:
 [[0.49935 0.50065]
 [0.51548 0.48452]]
emission matrix B:
 [[0.17617 0.16513 0.16303 0.16009 0.16353 0.17204]
 [0.17101 0.16197 0.17033 0.16747 0.16684 0.16237]]
initial distribution I:
 [0.5 0.5]


In [17]:
def loss(model, y):
  alpha, _, lastcol, lik = model(y)
  L = -tf.reduce_mean(tf.math.log(lik))
  return L

L = loss(F, inputs)
#print(f"likelihoods = {lik}\nloss (avg neg log lik)= {loss}")
print("Loss test: {}".format(L))

Loss test: 21.50402069091797


In [18]:
def grad(model, inputs):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [19]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

In [20]:
model = F
loss_value, grads = grad(model, inputs)

print("Step: {}, Initial Loss: {}".format(opt.iterations.numpy(),
                                          loss_value.numpy()))


Step: 0, Initial Loss: 21.50402069091797


In [21]:
# Keep results for plotting
train_loss_results = []

num_epochs = 201
m = 32 # training batches

for epoch in range(num_epochs):
  epoch_loss_avg = tf.keras.metrics.Mean()
  epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

  # Training loop - using batches
  for y in ds.take(m):
    # Optimize the model
    loss_value, grads = grad(model, y)
    opt.apply_gradients(zip(grads, model.trainable_variables))

    # Track progress
    epoch_loss_avg.update_state(loss_value)  # Add current batch loss

    # End epoch
  train_loss_results.append(epoch_loss_avg.result())

  if epoch % 1 == 0:
    print("Epoch {:03d}: Loss: {:.3f}".format(epoch, epoch_loss_avg.result()))
  if epoch % 10 == 0:
    print_pars(dcc)

Epoch 000: Loss: 21.427
transition matrix A:
 [[0.60341 0.39659]
 [0.60875 0.39125]]
emission matrix B:
 [[0.15669 0.15583 0.15121 0.15254 0.15548 0.22825]
 [0.15481 0.1545  0.15858 0.16032 0.16056 0.21123]]
initial distribution I:
 [0.3965 0.6035]
Epoch 001: Loss: 21.395
Epoch 002: Loss: 21.387
Epoch 003: Loss: 21.389
Epoch 004: Loss: 21.375
Epoch 005: Loss: 21.364
Epoch 006: Loss: 21.352
Epoch 007: Loss: 21.327
Epoch 008: Loss: 21.347
Epoch 009: Loss: 21.352
Epoch 010: Loss: 21.356
transition matrix A:
 [[0.88031 0.11969]
 [0.09572 0.90428]]
emission matrix B:
 [[0.12781 0.11884 0.12697 0.1186  0.12585 0.38192]
 [0.16649 0.17684 0.17013 0.17013 0.17805 0.13836]]
initial distribution I:
 [0.10255 0.89745]
Epoch 011: Loss: 21.352
Epoch 012: Loss: 21.349
Epoch 013: Loss: 21.358
Epoch 014: Loss: 21.336
Epoch 015: Loss: 21.333
Epoch 016: Loss: 21.329
Epoch 017: Loss: 21.337
Epoch 018: Loss: 21.332
Epoch 019: Loss: 21.332
Epoch 020: Loss: 21.338
transition matrix A:
 [[0.84935 0.15065]
 [0

In [22]:
print_pars(dcc)

transition matrix A:
 [[0.83965 0.16035]
 [0.04768 0.95232]]
emission matrix B:
 [[0.10481 0.09599 0.09516 0.10379 0.09706 0.50318]
 [0.16546 0.16977 0.16908 0.16476 0.16592 0.16502]]
initial distribution I:
 [0.00176 0.99824]


In [23]:
L = loss(F, inputs)

In [24]:
L

<tf.Tensor: shape=(), dtype=float32, numpy=21.334452>